In [2]:
from classes.SAMMed2DClass import SAMMed2DInferer
from utils.base_classes import Points
import utils.analysis as anUt
import utils.prompt as prUt
from utils.image import read_im_gt
from utils.interactivity import iterate_2d

import numpy as np

# Obtain model
device = 'cuda'
sammed2d_checkpoint_path = "/home/t722s/Desktop/UniversalModels/TrainedModels/sam-med2d_b.pth"
sammed2d_inferer = SAMMed2DInferer(sammed2d_checkpoint_path, device)


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.8). Upgrade using: pip install --upgrade albumentations


True
*******load /home/t722s/Desktop/UniversalModels/TrainedModels/sam-med2d_b.pth


In [3]:
# Obtain image, gt
img_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/imagesTr/BDMAP_00000001_0000.nii.gz'
gt_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/labelsTr/BDMAP_00000001.nii.gz'
img, gt = read_im_gt(img_path, gt_path, 6)


In [27]:
# Experiment: n randomly sampled points from foreground
seed = 11121
n_clicks = 1
# n_clicks = 5
point_prompt = prUt.get_pos_clicks2D_row_major(gt, n_clicks, seed = seed)
segmentation = sammed2d_inferer.predict(img, point_prompt)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 25/25 [00:00<00:00, 154.09it/s]


0.7081120204888369

In [3]:
# Experiment: 2d bounding box per slice with foreground
box_prompt = prUt.get_minimal_boxes_row_major(gt, 3, 3)
segmentation = sammed2d_inferer.predict(img, box_prompt)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 154.70it/s]


0.9396739793941389

In [6]:
# Experiment: get 3d bbox and slice it to feed it in 2d axially
box_prompt = prUt.get_bbox3d_sliced(gt)

segmentation = sammed2d_inferer.predict(img, box_prompt)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 25/25 [00:00<00:00, 167.42it/s]


0.5055264526156306

In [7]:
# Experiment: line interpolation
n_slices = 5
interpolation = 'linear'
point_prompt = prUt.line_interpolation(gt, n_slices, interpolation)

segmentation = sammed2d_inferer.predict(img, point_prompt)
anUt.compute_dice(segmentation, gt)

Using previously generated image embeddings


Performing inference on slices: 100%|██████████| 45/45 [00:00<00:00, 156.65it/s]


0.9143276197755548

In [3]:
# Experiment: box interpolation
n_boxes = 5

seed_boxes = prUt.get_seed_boxes(gt, n_boxes)
box_prompt = prUt.box_interpolation(seed_boxes)
segmentation = sammed2d_inferer.predict(img, box_prompt)
anUt.compute_dice(segmentation, gt)

Performing inference on slices: 100%|██████████| 25/25 [00:01<00:00, 19.10it/s]


0.7249539268193499

In [6]:
# Experiment: Point propagation
seed = 11121
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.point_propagation(sammed2d_inferer, img, seed_point, slices_to_infer, seed, n_clicks)
print(anUt.compute_dice(gt,segmentation))


Propagating up: 100%|██████████| 12/12 [00:00<00:00, 121.26it/s]


0.6715563146789912


In [ ]:
# Iteratively improve previous segmentation (from point propagation)
seed = 11121
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.point_propagation(sammed2d_inferer, img, seed_point, slices_to_infer, seed, n_clicks)

initial_prompt = prompt
condition = 'dof'
dof_bound = 90
dice_scores, dof, segmentations, prompts = iterate_2d(sammed2d_inferer, img, gt, segmentation, initial_prompt, sammed2d_inferer.pass_prev_prompts,
                                                                         condition = 'dof', init_dof = 5, dof_bound = dof_bound, seed = seed, detailed = True)

In [8]:
# Experiment: Box propagation

seed_box = prUt.get_seed_box(gt)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, prompt = prUt.box_propagation(sammed2d_inferer, img, seed_box, slices_to_infer)

print(anUt.compute_dice(gt,segmentation))

Propagating up: 100%|██████████| 12/12 [00:00<00:00, 66.97it/s]


0.5264982538428203


In [1]:
# Experiment: interaction to performance, start with points per slice DEPRECATED - only iterates on a slice at a time

seed = 11121
n_seed_clicks = 1 # For initial segmentation
fix_worst_slice = False

target_performance = 0.95

## Generate initial segmentation to improve
np.random.seed(seed)
point_prompt = prUt.get_pos_clicks2D_row_major(gt, n_seed_clicks, seed = seed)
segmentation = sammed2d_inferer.predict(img, point_prompt)

initial_dof = len(point_prompt.coords)*2+2
dice, revised_slices, dof, segmentations = prUt.iter_improve_perf_sammed2d(img, gt, segmentation, sammed2d_inferer, 
                                                                           point_prompt, initial_dof = initial_dof, target_performance = target_performance,
                                                                           fix_worst_slice = fix_worst_slice, seed = None)
dice


NameError: name 'np' is not defined